# Feature Engineering

In [ ]:
#Importing Libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading Data

In [ ]:
# reading preprocessed data
train_sales=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/preprocessed_train.csv")
items=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/preprocessed_items.csv")
item_categories=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/preprocessed_item_categories.csv")
shops=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/preprocessed_shops.csv")

**Monthly Sales Data**

In [ ]:
monthly_sales_data=pd.DataFrame(train_sales.groupby(["date_block_num","shop_id","item_id"])["item_cnt_day"].apply(np.sum)).reset_index()
monthly_sales_data.columns=['date_block_num','shop_id','item_id','item_cnt_month']

In [ ]:
# clipping item_cnt_month into [0,20]
monthly_sales_data['item_cnt_month']=monthly_sales_data['item_cnt_month'].clip(0,20)

In [ ]:
monthly_sales_data.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


**Reading Test Data**

In [ ]:
test=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/test.csv")

In [ ]:
test.drop('ID', axis=1, inplace=True)

test['date_block_num']=34
#merging train data with test data
data= pd.concat([monthly_sales_data,test],ignore_index=True,sort=False,keys= ['date_block_num','shop_id','item_id'])

**Adding Month and Year Features**

In [ ]:
def fun(date):
  if date<12:
    return 2013
  elif date>11 and date<24:
    return 2014
  else:
    return 2015

data['year']=data['date_block_num'].map(fun)

In [ ]:
data['month']=(data['date_block_num']%12)+1

# adding feature for december or not as december month as peak in sales
data['is_december']=data['month'].apply(lambda x:1 if x==12 else 0)

In [ ]:
data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december
0,0,0,32,6.0,2013,1,0
1,0,0,33,3.0,2013,1,0
2,0,0,35,1.0,2013,1,0
3,0,0,43,1.0,2013,1,0
4,0,0,51,2.0,2013,1,0


In [ ]:
#merging all  preprocessed data shops,items,item_categories
data=pd.merge(data,shops,how='left',on=['shop_id'])
data=pd.merge(data,items,how='left',on=['item_id'])
data=pd.merge(data,item_categories,how='left',on=['item_category_id'])

data.drop('item_name', axis=1, inplace=True) #dropping item_name column
data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december,shop_city,shop_category,item_category_id,item_category,item_sub_category
0,0,0,32,6.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD
1,0,0,33,3.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,37,Movie,Blu-Ray
2,0,0,35,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD
3,0,0,43,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD
4,0,0,51,2.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,57,Music,MP3


**Label Encoding categorical features shop_city, shop_category, item_category, and item_sub_category**

In [ ]:
le=preprocessing.LabelEncoder()

data['shop_city_encoding']=le.fit_transform(data['shop_city'])              # Label Encoding shop_city
data['shop_category_encoding']=le.fit_transform(data['shop_category'])      # shop_category
data['item_category_encoding']=le.fit_transform(data['item_category'])       # item_category
data['item_sub_category_encoding']=le.fit_transform(data['item_sub_category']) # and item_sub_category

In [ ]:
data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december,shop_city,shop_category,item_category_id,item_category,item_sub_category,shop_city_encoding,shop_category_encoding,item_category_encoding,item_sub_category_encoding
0,0,0,32,6.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD,26,6,5,19
1,0,0,33,3.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,37,Movie,Blu-Ray,26,6,11,8
2,0,0,35,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD,26,6,5,19
3,0,0,43,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,Cinema,DVD,26,6,5,19
4,0,0,51,2.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,57,Music,MP3,26,6,12,38


**Adding Revenue Feature**

In [ ]:
# calculating revenue by multiplying item_price and item_cnt_day
train_sales['revenue']=train_sales['item_price']*train_sales['item_cnt_day']

# calculating revenue for each shop_id and each item_id combination in month
revenue=pd.DataFrame(train_sales.groupby(["date_block_num","shop_id","item_id"])["revenue"].apply(np.sum)).reset_index()
  
# calculating the lag for revenue feature
temp=revenue.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_id','item_id','revenue_lag']

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.merge.html
temp1=pd.merge(revenue,temp,how='left',on=["date_block_num","shop_id","item_id"])
data=pd.merge(data,temp1,how='left',on=["date_block_num","shop_id","item_id"])
temp1.head(2)

,date_block_num,shop_id,item_id,revenue,revenue_lag
0,0,0,32,1326.0,NaN
1,0,0,33,1041.0,NaN


**Revenue related features**

In [ ]:
#merging all  preprocessed data train_sales,shops,items,item_categories
preprocessed_data=train_sales.merge(shops)
preprocessed_data=preprocessed_data.merge(items)
preprocessed_data=preprocessed_data.merge(item_categories)

preprocessed_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,revenue,shop_city,shop_category,item_name,item_category_id,item_category,item_sub_category
0,0,59,22154,999.0,1.0,1,2013,999.0,Yaroslavl,shopping_center_Altair,PHENOMENON_2012_BD,37,Movie,Blu-Ray
1,0,25,22154,999.0,1.0,1,2013,999.0,Moscow,TRK_Atrium,PHENOMENON_2012_BD,37,Movie,Blu-Ray
2,0,25,22154,999.0,1.0,1,2013,999.0,Moscow,TRK_Atrium,PHENOMENON_2012_BD,37,Movie,Blu-Ray
3,0,25,22154,999.0,1.0,1,2013,999.0,Moscow,TRK_Atrium,PHENOMENON_2012_BD,37,Movie,Blu-Ray
4,0,25,22154,999.0,1.0,1,2013,999.0,Moscow,TRK_Atrium,PHENOMENON_2012_BD,37,Movie,Blu-Ray


In [ ]:
preprocessed_data['revenue']=preprocessed_data['item_price']*preprocessed_data['item_cnt_day']

#adding shop_revenue feature by aggregating revenue of each shop_id in a month
shop_revenue=pd.DataFrame(preprocessed_data.groupby(["date_block_num","shop_id"])["revenue"].apply(np.sum)).reset_index()
shop_revenue.columns=["date_block_num","shop_id","shop_revenue"]

# adding the lag feature for shop_revenue
temp=shop_revenue.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_id','shop_revenue_lag']

In [ ]:
temp1=pd.merge(shop_revenue,temp,how='left',on=["date_block_num","shop_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','shop_id'])
temp1.head()

,date_block_num,shop_id,shop_revenue,shop_revenue_lag
0,0,0,2966412.00,NaN
1,0,1,1527320.00,NaN
2,0,2,1085064.80,NaN
3,0,3,560585.01,NaN
4,0,4,1433666.00,NaN


In [ ]:
# adding shop_itemcategory_revenue feature by aggregating revenue of each shop_id and each item_category_id combination in month
shop_itemcategory_revenue=pd.DataFrame(preprocessed_data.groupby(["date_block_num","shop_id","item_category_id"])["revenue"].apply(np.sum)).reset_index()
shop_itemcategory_revenue.columns=["date_block_num","shop_id","item_category_id","shop_itemcategory_revenue"]

# adding the lag feature for shop_itemcategory_revenue
temp=shop_itemcategory_revenue.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_id','item_category_id','shop_itemcategory_revenue_lag']

In [ ]:
temp1=pd.merge(shop_itemcategory_revenue,temp,how='left',on=["date_block_num","shop_id","item_category_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','shop_id','item_category_id'])
temp1.head(2)


,date_block_num,shop_id,item_category_id,shop_itemcategory_revenue,shop_itemcategory_revenue_lag
0,0,0,2,99380.0,NaN
1,0,0,3,6776.0,NaN


**Price related Features**

In [ ]:
# adding  date_item_avgprice feature by averaging item_price of each item_id in a month
date_item_avgprice=pd.DataFrame(preprocessed_data.groupby(["date_block_num","item_id"])["item_price"].apply(np.mean)).reset_index()
date_item_avgprice.columns=["date_block_num","item_id","date_item_avgprice"]

# adding the lag feature for date_item_avgprice 
temp=date_item_avgprice.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','item_id','date_item_avgprice_lag']

In [ ]:
temp1=pd.merge(date_item_avgprice,temp,how='left',on=["date_block_num","item_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','item_id'])
temp1.head(2)

,date_block_num,item_id,date_item_avgprice,date_item_avgprice_lag
0,0,19,28.0,NaN
1,0,27,2325.0,NaN


In [ ]:
# adding  shop_item_avgprice feature by averaging item_price of each shop_id and each item_id combination in month
shop_item_avgprice=pd.DataFrame(preprocessed_data.groupby(["date_block_num","shop_id","item_id"])["item_price"].apply(np.mean)).reset_index()
shop_item_avgprice.columns=["date_block_num","shop_id","item_id","shop_item_avgprice"]

# adding the lag feature for shop_item_avgprice
temp=shop_item_avgprice.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_id','item_id','shop_item_avgprice_lag']

In [ ]:
temp1=pd.merge(shop_item_avgprice,temp,how='left',on=["date_block_num","shop_id","item_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','shop_id','item_id'])
temp1.head()

,date_block_num,shop_id,item_id,shop_item_avgprice,shop_item_avgprice_lag
0,0,0,32,221.0,NaN
1,0,0,33,347.0,NaN
2,0,0,35,247.0,NaN
3,0,0,43,221.0,NaN
4,0,0,51,128.5,NaN


In [ ]:
# adding  item_avgprice feature by averaging item_price of each item_id
item_avgprice=pd.DataFrame(preprocessed_data.groupby(["item_id"])["item_price"].apply(np.mean)).reset_index()
item_avgprice.columns=["item_id","item_avgprice"]
data=pd.merge(data,item_avgprice,how='left',on=["item_id"])
item_avgprice.head()

,item_id,item_avgprice
0,0,58.0
1,1,4490.0
2,2,58.0
3,3,79.0
4,4,58.0


In [ ]:
# adding delta_item_price i.e change in item_avgprice in current month to the item_avgprice in previous month
data['delta_item_price']=(data['date_item_avgprice_lag']-data['item_avgprice'])/data['item_avgprice']

In [ ]:
data.head(2)

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december,shop_city,shop_category,item_category_id,...,shop_revenue,shop_revenue_lag,shop_itemcategory_revenue,shop_itemcategory_revenue_lag,date_item_avgprice,date_item_avgprice_lag,shop_item_avgprice,shop_item_avgprice_lag,item_avgprice,delta_item_price
0,0,0,32,6.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,...,2966412.0,NaN,308301.0,NaN,338.110349,NaN,221.0,NaN,249.629240,NaN
1,0,0,33,3.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,37,...,2966412.0,NaN,93622.0,NaN,488.517241,NaN,347.0,NaN,252.831928,NaN


**Item Count related Features**

In [ ]:
# adding date_avg_item_cnt feature by averaging item_cnt_month of each month
date_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num"])["item_cnt_month"].apply(np.mean)).reset_index()
date_avg_item_cnt.columns=["date_block_num","date_avg_item_cnt"]

# adding the lag feature for  date_avg_item_cnt
temp=date_avg_item_cnt.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','date_avg_item_cnt_lag']

In [ ]:
temp1=pd.merge(date_avg_item_cnt,temp,how='left',on=["date_block_num"])
data=pd.merge(data,temp1,how='left',on=['date_block_num'])
temp1.head()

,date_block_num,date_avg_item_cnt,date_avg_item_cnt_lag
0,0,2.005235,NaN
1,1,2.033703,2.005235
2,2,2.122372,2.033703
3,3,1.888155,2.122372
4,4,1.867326,1.888155


In [ ]:
# adding date_shop_avg_item_cnt feature by averaging item_cnt_month of each shop_id in month
date_shop_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","shop_id"])["item_cnt_month"].apply(np.mean)).reset_index()
date_shop_avg_item_cnt.columns=["date_block_num","shop_id","date_shop_avg_item_cnt"]

#calculating lags 1,2,3 for date_shop_avg_item_cnt. through modeling found first 3 lags were useful
datashopavgitemcnt=date_shop_avg_item_cnt
lags=[1,2,3]
for i in lags:
  temp=date_shop_avg_item_cnt.copy()
  temp['date_block_num']+=i
  temp.columns=['date_block_num','shop_id','date_shop_avg_item_cnt_lag_'+str(i)]
  datashopavgitemcnt=pd.merge(datashopavgitemcnt,temp,how='left',on=['date_block_num','shop_id'])

In [ ]:
#merging  date_shop_avg_item_cnt feature and its lag features with data
data=pd.merge(data,datashopavgitemcnt,how='left',on=['date_block_num','shop_id'])
datashopavgitemcnt.head(2)

,date_block_num,shop_id,date_shop_avg_item_cnt,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3
0,0,0,2.268763,NaN,NaN,NaN
1,0,1,1.910098,NaN,NaN,NaN


In [ ]:
# adding date_item_avg_item_cnt feature by averaging item_cnt_month of each item_id in month
date_item_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","item_id"])["item_cnt_month"].apply(np.mean)).reset_index()
date_item_avg_item_cnt.columns=["date_block_num","item_id","date_item_avg_item_cnt"]

#calculating lags 1,2,3 for date_item_avg_item_cnt. through modeling found first 3 lags were useful
dataitemavgitemcnt=date_item_avg_item_cnt
lags=[1,2,3]
for i in lags:
  temp=date_item_avg_item_cnt.copy()
  temp['date_block_num']+=i
  temp.columns=['date_block_num','item_id','date_item_avg_item_cnt_lag_'+str(i)]
  dataitemavgitemcnt=pd.merge(dataitemavgitemcnt,temp,how='left',on=['date_block_num','item_id'])

In [ ]:
# merging  date_item_avg_item_cnt feature and its lag features with data
data=pd.merge(data,dataitemavgitemcnt,how='left',on=['date_block_num','item_id'])
dataitemavgitemcnt.head(2)

,date_block_num,item_id,date_item_avg_item_cnt,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3
0,0,19,1.0,NaN,NaN,NaN
1,0,27,1.0,NaN,NaN,NaN


In [ ]:
# adding date_itemcategory_avg_item_cnt feature by averaging item_cnt_month of each item_category_id in month
date_itemcategory_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","item_category_id"])["item_cnt_month"].apply(np.mean)).reset_index()
date_itemcategory_avg_item_cnt.columns=["date_block_num","item_category_id","date_itemcategory_avg_item_cnt"]

# adding the lag feature for date_itemcategory_avg_item_cnt
temp=date_itemcategory_avg_item_cnt.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','item_category_id','date_itemcategory_avg_item_cnt_lag']

In [ ]:
temp1=pd.merge(date_itemcategory_avg_item_cnt,temp,how='left',on=["date_block_num","item_category_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','item_category_id'])
temp1.head(2)

,date_block_num,item_category_id,date_itemcategory_avg_item_cnt,date_itemcategory_avg_item_cnt_lag
0,0,0,1.0,NaN
1,0,1,1.0,NaN


In [ ]:
# adding date_shop_itemcategory_avg_item_cnt feature by averaging item_cnt_month of each shop_id and each item_category_id combination in month
date_shop_itemcategory_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","shop_id","item_category_id"])["item_cnt_month"].apply(np.mean)).reset_index()
date_shop_itemcategory_avg_item_cnt.columns=["date_block_num","shop_id","item_category_id","date_shop_itemcategory_avg_item_cnt"]

# adding the lag feature for date_shop_itemcategory_avg_item_cnt
temp=date_shop_itemcategory_avg_item_cnt.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_id','item_category_id','date_shop_itemcategory_avg_item_cnt_lag']

In [ ]:
temp1=pd.merge(date_shop_itemcategory_avg_item_cnt,temp,how='left',on=["date_block_num","shop_id","item_category_id"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','shop_id','item_category_id'])
temp1.head(2)

,date_block_num,shop_id,item_category_id,date_shop_itemcategory_avg_item_cnt,date_shop_itemcategory_avg_item_cnt_lag
0,0,0,2,3.3125,NaN
1,0,0,3,20.0000,NaN


In [ ]:
# adding date_city_avg_item_cnt feature by averaging item_cnt_month of each shop_city in month
date_city_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","shop_city"])["item_cnt_month"].apply(np.mean)).reset_index()
date_city_avg_item_cnt.columns=["date_block_num","shop_city","date_city_avg_item_cnt"]

# adding the lag feature for date_city_avg_item_cnt
temp=date_city_avg_item_cnt.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','shop_city','date_city_avg_item_cnt_lag']

In [ ]:
temp1=pd.merge(date_city_avg_item_cnt,temp,how='left',on=["date_block_num","shop_city"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','shop_city'])
temp1.head()

,date_block_num,shop_city,date_city_avg_item_cnt,date_city_avg_item_cnt_lag
0,0,Adygea,1.574176,NaN
1,0,Balashikha,1.409926,NaN
2,0,Chekhov,1.820862,NaN
3,0,Kaluga,2.123577,NaN
4,0,Kazan,1.622087,NaN


In [ ]:
# adding date_item_city_avg_item_cnt feature by averaging item_cnt_month of each item_id and each shop_city combination in month
date_item_city_avg_item_cnt=pd.DataFrame(data.groupby(["date_block_num","item_id","shop_city"])["item_cnt_month"].apply(np.mean)).reset_index()
date_item_city_avg_item_cnt.columns=["date_block_num","item_id","shop_city","date_item_city_avg_item_cnt"]

# adding the lag feature for date_item_city_avg_item_cnt
temp=date_item_city_avg_item_cnt.copy()
temp['date_block_num']+=1
temp.columns=['date_block_num','item_id','shop_city','date_item_city_avg_item_cnt_lag']

In [ ]:
temp1=pd.merge(date_item_city_avg_item_cnt,temp,how='left',on=["date_block_num","item_id","shop_city"])
data=pd.merge(data,temp1,how='left',on=['date_block_num','item_id','shop_city'])
temp1.head(2)

,date_block_num,item_id,shop_city,date_item_city_avg_item_cnt,date_item_city_avg_item_cnt_lag
0,0,19,Moscow,1.0,NaN
1,0,27,Adygea,1.0,NaN


**Calculating lag features for item_cnt_month**

In [ ]:
#calculating lags 1,2,3 for item_cnt_month. through modeling found first 3 lags were useful
monthly_data=monthly_sales_data
lags=[1,2,3]
for i in lags:
  temp=monthly_sales_data.copy()
  temp['date_block_num']+=i
  temp.columns=['date_block_num','shop_id','item_id','item_cnt_month_lag_'+str(i)]
  monthly_data=pd.merge(monthly_data,temp,how='left',on=['date_block_num','shop_id','item_id'])

monthly_data.head(2)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,0,32,6.0,NaN,NaN,NaN
1,0,0,33,3.0,NaN,NaN,NaN


In [ ]:
#merging item_cnt_month lag features with data
monthly_data.drop('item_cnt_month',axis=1,inplace=True)
monthly_data=pd.merge(data,monthly_data,how='left',on=['date_block_num','shop_id','item_id'])

monthly_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december,shop_city,shop_category,item_category_id,...,date_itemcategory_avg_item_cnt_lag,date_shop_itemcategory_avg_item_cnt,date_shop_itemcategory_avg_item_cnt_lag,date_city_avg_item_cnt,date_city_avg_item_cnt_lag,date_item_city_avg_item_cnt,date_item_city_avg_item_cnt_lag,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,0,32,6.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,...,NaN,2.126050,NaN,2.128316,NaN,6.5,NaN,NaN,NaN,NaN
1,0,0,33,3.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,37,...,NaN,1.440191,NaN,2.128316,NaN,3.0,NaN,NaN,NaN,NaN
2,0,0,35,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,...,NaN,2.126050,NaN,2.128316,NaN,1.0,NaN,NaN,NaN,NaN
3,0,0,43,1.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,40,...,NaN,2.126050,NaN,2.128316,NaN,1.0,NaN,NaN,NaN,NaN
4,0,0,51,2.0,2013,1,0,Yakutsk,Ordzhonikidze_56_francs,57,...,NaN,1.176471,NaN,2.128316,NaN,2.0,NaN,NaN,NaN,NaN


In [ ]:
#removing first 3 date_block_nums having null values of lag features item_cnt_month
monthly_data=monthly_data[monthly_data['date_block_num']>2]

In [ ]:
# dropping label encoded categorical features
monthly_data.drop(['shop_city','shop_category','item_category','item_sub_category'],axis=1, inplace=True)

In [ ]:
monthly_data['item_cnt_month'].fillna(0,inplace=True)

In [ ]:
pickle.dump(monthly_data,open("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/data_with_nulls.p","wb"))

In [ ]:
# checking for null values in final data
monthly_data.isnull().sum()

date_block_num                                   0
shop_id                                          0
item_id                                          0
item_cnt_month                                   0
year                                             0
month                                            0
is_december                                      0
item_category_id                                 0
shop_city_encoding                               0
shop_category_encoding                           0
item_category_encoding                           0
item_sub_category_encoding                       0
revenue                                     214200
revenue_lag                                1009264
shop_revenue                                214200
shop_revenue_lag                            218907
shop_itemcategory_revenue                   214200
shop_itemcategory_revenue_lag               226774
date_item_avgprice                          214200
date_item_avgprice_lag         

There are null values for some features in test data as the test data has only shop_id and item_id but not have item_price.and there are some null values due to the lag features i.e particular shop_id or item_id not have feature value in previous month. Tried to impute null values through mean, median and MICE Imputer. But found model performing better when null values imputed though MICE.

**Filling null values in data using MICE Imputer**

In [ ]:
!pip install fancyimpute

     |████████████████████████████████| 154 kB 15.8 MB/s 
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=d392a5a507a4b066cb3feea2f3978a1e7bf81ad509c17fbf49c43ae0d95dcaeb
  Stored in directory: /root/.cache/pip/wheels/e3/04/06/a1a7d89ef4e631ce6268ea2d8cde04f7290651c1ff1025ce68
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=e35892e7cf24af7696b16697eeac3cb0343aa7b48bd8f56219b80eae35e65807
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built fancyimpute knnimpute


In [ ]:
from fancyimpute import IterativeImputer

mice_imputer = IterativeImputer()
#imputing the null values with MICE imputer
monthly_data=mice_imputer.fit_transform(monthly_data)

In [ ]:
monthly_data=pd.DataFrame(monthly_data,columns=['date_block_num','shop_id','item_id','item_cnt_month','year','month','is_december','item_category_id','shop_city_encoding',
                                                'shop_category_encoding','item_category_encoding','item_sub_category_encoding','revenue','revenue_lag','shop_revenue',
                                                'shop_revenue_lag','shop_itemcategory_revenue','shop_itemcategory_revenue_lag','date_item_avgprice','date_item_avgprice_lag',
                                                'shop_item_avgprice','shop_item_avgprice_lag','item_avgprice','delta_item_price','date_avg_item_cnt','date_avg_item_cnt_lag',
                                                'date_shop_avg_item_cnt','date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3',
                                                'date_item_avg_item_cnt','date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3',
                                                'date_itemcategory_avg_item_cnt','date_itemcategory_avg_item_cnt_lag','date_shop_itemcategory_avg_item_cnt',
                                                'date_shop_itemcategory_avg_item_cnt_lag','date_city_avg_item_cnt','date_city_avg_item_cnt_lag','date_item_city_avg_item_cnt',
                                                'date_item_city_avg_item_cnt_lag','item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month_lag_3'])

In [ ]:
monthly_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,is_december,item_category_id,shop_city_encoding,shop_category_encoding,...,date_itemcategory_avg_item_cnt_lag,date_shop_itemcategory_avg_item_cnt,date_shop_itemcategory_avg_item_cnt_lag,date_city_avg_item_cnt,date_city_avg_item_cnt_lag,date_item_city_avg_item_cnt,date_item_city_avg_item_cnt_lag,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,3.0,2.0,31.0,1.0,2013.0,4.0,0.0,37.0,0.0,46.0,...,1.568948,1.133333,1.071429,1.455224,1.754673,1.0,1.000000,1.000000,4.000000,5.787032
1,3.0,2.0,464.0,1.0,2013.0,4.0,0.0,49.0,0.0,46.0,...,2.103448,1.166667,2.600000,1.455224,1.754673,1.0,1.667326,2.122765,1.066820,0.885321
2,3.0,2.0,482.0,1.0,2013.0,4.0,0.0,73.0,0.0,46.0,...,2.606272,1.666667,1.285714,1.455224,1.754673,1.0,1.000000,1.000000,1.000000,1.000000
3,3.0,2.0,484.0,3.0,2013.0,4.0,0.0,73.0,0.0,46.0,...,2.606272,1.666667,1.285714,1.455224,1.754673,3.0,2.000000,2.000000,4.000000,2.000000
4,3.0,2.0,485.0,2.0,2013.0,4.0,0.0,73.0,0.0,46.0,...,2.606272,1.666667,1.285714,1.455224,1.754673,2.0,2.029695,1.720840,1.590524,1.822070


In [ ]:
#saving the data in a pickle file
pickle.dump(monthly_data,open("/content/drive/MyDrive/Colab Notebooks/Predict_Future_Sales/data.p","wb"))